In [ ]:
import pandas as pd
import numpy as np
# New import
import requests
import datetime

In [ ]:
file_name = "data-txn-price-t36.json"
# Change relative file directory to match where the data is
data_path = "././data/"
data_directory = data_path + file_name
data = pd.read_json(data_directory)

### Change The Below Variables

In [ ]:
# Variables to change for function
price_offset_in_days = 0

# Prefer that you follow the format "{Ticker/SPY} Price # {Days/Months/etc} After Txn Date"
ticker_column_name = ""
spy_column_name = ""

output_file_name = "youChange.json" # need extension ".json"

## Run the Cell Below to grab the data. This will take ~40 minutes

In [ ]:
data['Transaction Date'] = pd.to_datetime(data['Transaction Date'])
data['Ticker'] = data['Ticker'].str.upper()
groups = data.groupby('Ticker')
bad_tickers = []


for ticker, tData in groups:
    
# test ticker
    test_link = f"https://finance.yahoo.com/quote/{ticker}?p={ticker}&.tsrc=fin-srch"
    val = requests.get(test_link).status_code
    if val == 404: # This ticker is invalid, so continue
        bad_tickers.append((ticker,"invalid ticker"))
        continue
# alg
    try:
        min_date = tData['Transaction Date'].min() + datetime.timedelta(days=price_offset_in_days)
        max_date = tData['Transaction Date'].max() + datetime.timedelta(days=1+price_offset_in_days)
        start_date = str(int(min_date.timestamp()))
        end_date = str(int(max_date.timestamp()))
    except:
        bad_tickers.append((ticker,"ticker date"))
        continue
    
    interval = '1d'
    event='history'
    url = 'https://query1.finance.yahoo.com/v7/finance/download/'      + ticker + '?period1=' + start_date + '&period2=' + end_date + '&interval='      + interval + '&events=' + event
    spy_url = 'https://query1.finance.yahoo.com/v7/finance/download/'      + "SPY" + '?period1=' + start_date + '&period2=' + end_date + '&interval='      + interval + '&events=' + event
    
    try:
        b_data = pd.read_csv(url)
        b_data['Date'] = pd.to_datetime(b_data['Date'])
        spy_data = pd.read_csv(spy_url)
        spy_data['Date'] = pd.to_datetime(spy_data['Date'])
    except:
        b_data = []
        spy_data = []
        bad_tickers.append((ticker,"data retrieval"))
    # now, get the data from yahoo prices
    tData = tData.reset_index()
    for i in range(tData.shape[0]):
        current_txn_date = tData.loc[i,'Transaction Date']
        lower_date = current_txn_date + datetime.timedelta(days=price_offset_in_days) - datetime.timedelta(days=2)
        upper_date = current_txn_date + datetime.timedelta(days=2+price_offset_in_days)
        try:
            b_mask = (b_data['Date'] >= lower_date) & (b_data['Date'] <= upper_date)
            spy_mask = (spy_data['Date'] >= lower_date) & (spy_data['Date'] <= upper_date)
            data.loc[(data['Transaction Date'] == current_txn_date)&(data['Ticker']==ticker),ticker_column_name] = b_data[b_mask]['High'].mean()
            data.loc[(data['Transaction Date'] == current_txn_date)&(data['Ticker']==ticker),spy_column_name] = spy_data[spy_mask]['High'].mean()
        except:
            pass

In [ ]:
output_path = data_path + output_file_name
data.to_json(output_path)